# 7. 앙상블 학습과 랜덤 포레스트

일련의 예측기를 `앙상블`이라 부르며, 이를 `앙상블 학습`이라고 함. 예를 들어 결정 트리의 앙상블을 랜덤 포레스트라고 함.

## 7.1 투표 기반 분류기

- 각 분류기의 예측을 모아 가장 많이 선택된 클래스를 예측, `직접 투표` 분류기
- 약한 학습기도 모이면 강한 학습기가 되는데, 이는 큰 수의 법칙으로 설명할 수 있음
- 앙상블 방법은 **예측기가 독립적일 떄 최고의 성능**을 발휘
- 다양한 분류기를 얻는 방법은 **각기 다른 알고리즘으로 학습**시키는 것

투표 기반 분류기(VotingClassifier)

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting = 'hard'
)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

정확도 확인

In [3]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.888
SVC 0.896
VotingClassifier 0.904


**`voting = hard/soft`**

`hard`: 직접 투표  
`soft`: 간접 투표  

- 분류기가 predict_proba() 메서드가 있으면 예측을 평균내어 확률이 가장 높은 클래스를 예측
- 직접 투표 방식보다 성능이 높음

## 7.2 배깅과 페이스팅

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2FbCmIoE%2Fbtqy0RW3DBW%2Fcu6uK7BXlJ5KBUZSmz7R0K%2Fimg.png' width=70%></img>

**같은 알고리즘을 사용하되 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습**  
**`배깅`**: 중복 허용하여 샘플링  
**`페이스팅`**: 중복 허용 X  
  
- 배깅만이 한 예측기를 위해 같은 훈련 샘플을 여러 번 샘플링할 수 있음
- 수집 함수; 분류-최빈값(hard voting), 회귀-평균
- CPU 코어나 서버에서병렬로 학습 가능

### 7.2.1 사이킷런의 배깅과 페이스팅

- `bootstrap=False`: 페이스팅 사용
- `n_jobs`: 사용할 CPU 코어 수, -1로 지정하면 모든 코어 사용(디폴트 1)
- 배깅은 서브셋의 다양성을 증가시키고, 배깅이 페이스팅보다 더 나은 모델을 만들기 위해 사용

결정 트리 분류기 500개의 앙상블 훈련

In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators = 500,
    max_samples = 100, bootstrap = True, n_jobs = -1
)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

### 7.2.2 oob 평가

- **oob**: 선택되지 않은 훈련 샘플  
- 앙상블 평가는 각 샘플의 obb 평가를 평균하여 얻음
- `oob_score = True`를 배깅에서사용하여 obb 평가를 수행할 수 있음
- `oob_score_` 평가 점수 결과 저장

In [5]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators = 500,
    bootstrap = True, n_jobs = -1, oob_score = True
)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.8986666666666666

In [6]:
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.904

비슷한 정확도를 가지는 것 확인

`oob_decision_function_`: 각 훈련 샘플의 클래스 확률 반환

In [7]:
bag_clf.oob_decision_function_

array([[0.36021505, 0.63978495],
       [0.26203209, 0.73796791],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.04046243, 0.95953757],
       [0.32432432, 0.67567568],
       [0.00558659, 0.99441341],
       [0.98984772, 0.01015228],
       [0.98378378, 0.01621622],
       [0.7740113 , 0.2259887 ],
       [0.01098901, 0.98901099],
       [0.82125604, 0.17874396],
       [0.87209302, 0.12790698],
       [0.95906433, 0.04093567],
       [0.05617978, 0.94382022],
       [0.00520833, 0.99479167],
       [0.96610169, 0.03389831],
       [0.95783133, 0.04216867],
       [1.        , 0.        ],
       [0.01675978, 0.98324022],
       [0.37356322, 0.62643678],
       [0.91304348, 0.08695652],
       [1.        , 0.        ],
       [0.98958333, 0.01041667],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.59459459, 0.40540541],
       [0.

## 7.3 랜덤 패치와 랜덤 서브스페이스

특성 샘플링은 더 다양한 예측기를 만들고 편향을 늘리는 대신 분산을 낮춤
- `랜덤 패치 방식`: 훈련 특성/샘플 모두 샘플링
- `서브스페이스 방식`: 훈련 샘플을모두 사용하고 특성은 샘플링

`max_features`, `bootstrap_features` 매개변수로 조절  
샘플이 아닌 특성에 대한 샘플링  
고차원의 데이터셋을 다룰 때 유용

## 7.4 랜덤 포레스트

배깅을 적용한 결정 트리의 앙상블

- `max_samples`: 훈련 세트의 크기
- BaggingClassifier에 DecisionTreeClassifier 대신 최적화된 RandomForestClassifier 사용

트리의 노드를 분할할 때 최선의 특성을 찾는 대신 무작위 선택 후보 중 최적의 특성을 찾음  
편향을 손해보는 대신 분산을 낮춤  

In [8]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators = 500, max_leaf_nodes = 16, n_jobs = -1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

배깅분류를 사용하여 랜덤포레스트분류를 유사하게 만든 코드

In [9]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features = 'auto', max_leaf_nodes = 16),
    n_estimators = 500, max_samples = 1.0, bootstrap = True, n_jobs = -1)

### 7.4.1 엑스트라 트리

- 극단적으로 무작위한 트리의 랜덤 포레스트
- 최적의 임곗값을 찾는 대신 후보 특성을 사용해 무작위로 분할 후 최상의 분할을 선택
- `ExtraTreesClassifier` 사용

### 7.4.2 특성 중요도

- `feature_importances_`: 특성의 상대적 중요도 측정
- 평균적으로 불순도를 얼마나 감소시키는지 확인하여 측정

In [10]:
from sklearn.datasets import load_iris

iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators = 500, n_jobs = -1)
rnd_clf.fit(iris['data'], iris['target'])

for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.103308914095434
sepal width (cm) 0.02404215631644509
petal length (cm) 0.43007404064090343
petal width (cm) 0.44257488894721736


## 7.5 부스팅

- 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 기법
- 앞의 모델을 보완하면서 일련의 예측기 학습
- 단점: 병렬화 할 수 없음!

### 7.5.1 에이다부스트

- 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높여(업데이트) 이전 예측기 보완
- 가중치 합이 가장 큰 클래스가 예측 결과
- `AdaBoostClassifier`
- `learning rate`: 학습률 파라미터

In [11]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth = 1), n_estimators = 200,
    algorithm = 'SAMME.R', learning_rate = 0.5
)

ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### 7.5.2 그레디언트 부스팅

- 이전 예측기가 만든 **잔여 오차**에 새로운 예측기 학습
- `learning_rate`: 각 트리의 기여 정도, 낮으면 많은 트리가 필요하지만 예측 성능 좋아짐(규제-축소)

회귀결정트리로 학습해보기

In [12]:
import numpy as np

np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

In [13]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth = 2)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [14]:
y2 = y - tree_reg1.predict(X) #오차

tree_reg2 = DecisionTreeRegressor(max_depth = 2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

In [16]:
y3 = y2 - tree_reg2.predict(X)

tree_reg3 = DecisionTreeRegressor(max_depth = 2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

In [17]:
# 새로운 샘플에 대한 예측
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

NameError: name 'X_new' is not defined

앞선 코드와 같은 앙상블 코드

In [18]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators = 3, learning_rate = 1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

`staged_predict()`; 최적의 트리 수를 찾기 위한 조기 종료 기법  
훈련의 단계에서 예측기를 순회하는 반복자(iterator)를 반환

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators = 120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
         for y_pred in gbrt.staged_predict(X_val)]

bst_n_estimators = np.argmin(errors)+1

gbrt_best = GradientBoostingRegressor(max_depth = 2, n_estimators = bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=110)

`warm_start = True` fit이 호출될 때 기존 트리를 유지하고 훈련을 추가할 수 있음  
다음은 5번의 반복 동안 검증 오차가 향상되지 않을 시 중지

In [21]:
gbrt = GradientBoostingRegressor(max_depth = 2, warm_start = True)

min_val_error = float('inf')
error_ging_up = 0

for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    
    else:
        error_going_up += 1
        if error_going_up == 5:
            break #조기 종료

### 확률적 그레이디언트 부스팅

- `subsample`로 훈련 샘플의 비율 지정 가능
- 편향↑ 분산↓, 훈련 속도를 높임
- XGBoost가 유명

In [23]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

XGBoost는 자동 조기 종료도 지원

In [25]:
xgb_reg.fit(X_train, y_train, eval_set = [(X_val, y_val)], early_stopping_rounds = 2)
y_pred = xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.19989
[1]	validation_0-rmse:0.14641
[2]	validation_0-rmse:0.10740
[3]	validation_0-rmse:0.08246
[4]	validation_0-rmse:0.06677
[5]	validation_0-rmse:0.05907
[6]	validation_0-rmse:0.05475
[7]	validation_0-rmse:0.05229
[8]	validation_0-rmse:0.05125
[9]	validation_0-rmse:0.05094
[10]	validation_0-rmse:0.05128


## 7.6 스태킹

- 개별 알고리즘으로 예측한 데이터를 기반으로 다시 예측 수행
- 블렌더: 마지막 예측기
- 홀드 아웃 세트를 사용해 블렌더 학습